In [ ]:
import numpy as np #  algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt #Provides MATLAB-like plotting framework
import seaborn as sns #python visualization library
from sklearn.neighbors import NearestNeighbors
color = sns.color_palette()

train = pd.read_csv('../input/train_2016_v2.csv' , parse_dates=["transactiondate"])
prop = pd.read_csv('../input/properties_2016.csv', usecols=['longitude', 'latitude', 'parcelid', 'regionidzip',
                                                            'propertyzoningdesc', 'taxvaluedollarcnt', 'structuretaxvaluedollarcnt'])

In [ ]:
train = pd.merge(train, prop, on=['parcelid'], how='left')
train.sort_values('transactiondate', inplace=True)
train['latitude'] = train['latitude']/1e6
train['longitude'] = train['longitude']/1e6

In [ ]:
num_neighbors = 3
train_dates = train.transactiondate.unique()[1:]
# creating some columns that we will be populating
for n in range(1, num_neighbors+1):
        train['nn{}_logerror'.format(n)] = 0
        train['nn{}_distance'.format(n)] = 0
        train['nn{}_taxvaluedollarcnt'.format(n)] = 0
        train['nn{}_structuretaxvaluedollarcnt'.format(n)] = 0
        


In [ ]:
for d in train_dates:
    nbrs = NearestNeighbors(n_neighbors=num_neighbors, algorithm='ball_tree')\
            .fit(train.loc[train.transactiondate<d, ['longitude','latitude']])
    distances, indices = nbrs.kneighbors(train.loc[train.transactiondate==d, ['longitude','latitude']])
    indices = indices.reshape(-1, 1)
    
    nearest_logerrors = train.loc[indices.squeeze(), 'logerror']
    nearest_logerrors = nearest_logerrors.values.reshape(-1, num_neighbors)
    
    nearest_taxvaluedollarcnt = train.loc[indices.squeeze(), 'taxvaluedollarcnt']
    nearest_taxvaluedollarcnt = nearest_taxvaluedollarcnt.values.reshape(-1, num_neighbors)
    
    nearest_structuretaxvaluedollarcnt = train.loc[indices.squeeze(), 'structuretaxvaluedollarcnt']
    nearest_structuretaxvaluedollarcnt = nearest_structuretaxvaluedollarcnt.values.reshape(-1, num_neighbors)
    
    train.loc[train.transactiondate==d, ['nn1_logerror', 'nn2_logerror', 'nn3_logerror']] = nearest_logerrors
    train.loc[train.transactiondate==d, ['nn1_distance', 'nn2_distance', 'nn3_distance']] = distances
    train.loc[train.transactiondate==d, ['nn1_taxvaluedollarcnt', 'nn2_taxvaluedollarcnt', 'nn3_taxvaluedollarcnt']] = nearest_taxvaluedollarcnt
    train.loc[train.transactiondate==d, ['nn1_structuretaxvaluedollarcnt', 'nn2_structuretaxvaluedollarcnt', 'nn3_structuretaxvaluedollarcnt']] = nearest_structuretaxvaluedollarcnt
    


In [ ]:
b = train_dates

In [ ]:
train = b

In [ ]:
to_look = 10
fig,ax = plt.subplots(to_look, 1, figsize=(15, 15), sharex=True)
for i in range(to_look):
    sns.distplot(train.loc[train.regionidzip == train.regionidzip.value_counts().index[i], 'logerror'],
                 ax=ax[i], bins=80)
    ax[i].set_ylabel(train.regionidzip.value_counts().index[i])

In [ ]:
to_look = 10
fig,ax = plt.subplots(to_look, 1, figsize=(15, 15), sharex=True)
for i in range(to_look):
    sns.distplot(train.loc[train.propertyzoningdesc == train.propertyzoningdesc.value_counts().index[i], 'logerror'],
                 ax=ax[i], bins=80)
    ax[i].set_ylabel(train.propertyzoningdesc.value_counts().index[i])